In [19]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [20]:
def calculate_confidence_interval(data, window, min_periods, z_score_threshold):
    mean = data.rolling(window=window, min_periods=min_periods).mean()
    std = data.rolling(window=window, min_periods=min_periods).std()
    lower_bound = mean - z_score_threshold * std
    upper_bound = mean + z_score_threshold * std
    return pd.DataFrame({'LowerBound': lower_bound, 'UpperBound': upper_bound})

In [21]:
def calculate_iqr_outliers(data, multiplier):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    outliers = (data < lower_bound) | (data > upper_bound)
    return outliers

In [22]:
def plot_with_confidence_interval_and_forecast_outliers(actual_data, confidence_interval, forecast_data, forecast_outliers, title):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=actual_data.index, y=actual_data, mode='lines', name='Actual Data'))
    fig.add_trace(go.Scatter(x=confidence_interval.index.tolist() + confidence_interval.index.tolist()[::-1], 
                             y=confidence_interval['LowerBound'].tolist() + confidence_interval['UpperBound'].tolist()[::-1], 
                             fill='toself', fillcolor='rgba(255,0,0,0.2)', line=dict(color='rgba(255,0,0,0)'), hoverinfo="skip", showlegend=False))
    fig.add_trace(go.Scatter(x=forecast_data.index, y=forecast_data[~forecast_outliers], mode='markers', name='Forecast (Non-Outliers)', marker=dict(color='blue')))
    fig.add_trace(go.Scatter(x=forecast_data.index, y=forecast_data[forecast_outliers], mode='markers', name='Forecast (Outliers)', marker=dict(color='orange')))
    fig.update_layout(title=title, xaxis_title="Date", yaxis_title="Value", legend_title="Legend")
    fig.show()

In [23]:

# Load the actual data from the Excel file
actual_data = pd.read_excel('platform_3m_actuals_new.xlsx')

# Filter the data to only include the "8k chassis" platform
actual_data = actual_data[actual_data['Platform'] == 'NCS_Fixed - 4.8T']


import datetime

# Function to convert quarter-year to datetime
def convert_qtr_to_date(qtr):
    print(qtr)
    # Split the string into quarter and year
    qtr, year = qtr.split('FY')
    # Convert the quarter to month
    if qtr == 'Q1':
        month = 1
    elif qtr == 'Q2':
        month = 4
    elif qtr == 'Q3':
        month = 7
    elif qtr == 'Q4':
        month = 10
    
    # Convert the year to integer
    year = int(year)
    year = year + 2000
    return datetime.datetime(year, month,1)

# Convert the quarter-year to datetime
actual_data['qtr'] = actual_data['qtr'].apply(convert_qtr_to_date)

print(actual_data['qtr'])

Q3FY21
Q4FY21
Q1FY22
Q2FY22
Q3FY22
2409   2021-07-01
2410   2021-10-01
2411   2022-01-01
2412   2022-04-01
2413   2022-07-01
Name: qtr, dtype: datetime64[ns]


In [24]:
# Load the forecast data
forecast_data = pd.read_csv('mviai_platform_3m_forecast.csv')

# Filter the forecast data to include only the '8k_Chassis' platform
forecast_data = forecast_data[forecast_data['Platform'] == 'NCS_Fixed - 4.8T']

# Convert the quarter-year to datetime
forecast_data['qtr'] = forecast_data['qtr'].apply(convert_qtr_to_date)
print(forecast_data['qtr'])

Q1FY22
Q2FY22
Q3FY21
Q3FY22
Q4FY21
Q4FY22
604   2022-01-01
605   2022-04-01
606   2021-07-01
607   2022-07-01
608   2021-10-01
609   2022-10-01
Name: qtr, dtype: datetime64[ns]


In [25]:


# Filter the datasets to include only the quarters between years 2021 and 2022
actual_data = actual_data[(actual_data['qtr'].dt.year >= 2021) & (actual_data['qtr'].dt.year <= 2022)]
forecast_data = forecast_data[(forecast_data['qtr'].dt.year >= 2021) & (forecast_data['qtr'].dt.year <= 2022)]

# Set 'qtr' as the index
actual_data.set_index('qtr', inplace=True)
forecast_data.set_index('qtr', inplace=True)


#print(actual_data)
print(forecast_data)

                    Platform  Forecast_3m
qtr                                      
2022-01-01  NCS_Fixed - 4.8T           84
2022-04-01  NCS_Fixed - 4.8T          104
2021-07-01  NCS_Fixed - 4.8T     13899467
2022-07-01  NCS_Fixed - 4.8T          102
2021-10-01  NCS_Fixed - 4.8T        36130
2022-10-01  NCS_Fixed - 4.8T          102


In [26]:
# Calculate the confidence interval for the 'Bookings_3m' column in the actual data
confidence_interval = calculate_confidence_interval(actual_data['Bookings_3m'], window=3, min_periods=1, z_score_threshold=3)

# Calculate outliers in the forecast data
forecast_outliers = calculate_iqr_outliers(forecast_data['Forecast_3m'], multiplier=1.5)
print(forecast_outliers)

#print the outliers if they are true
print(forecast_data[forecast_outliers])

qtr
2022-01-01    False
2022-04-01    False
2021-07-01    False
2022-07-01    False
2021-10-01    False
2022-10-01    False
Name: Forecast_3m, dtype: bool
Empty DataFrame
Columns: [Platform, Forecast_3m]
Index: []


In [27]:
# Call the function to create the plot
plot_with_confidence_interval_and_forecast_outliers(
    actual_data=actual_data['Bookings_3m'], 
    confidence_interval=confidence_interval, 
    forecast_data=forecast_data['Forecast_3m'], 
    forecast_outliers=forecast_outliers, 
    title='8k Chassis Bookings_3m with Confidence Interval and Forecast Outliers'
)